In [86]:
# Added libraries.
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 60)

spark = SparkSession.builder.master("local[*]")\
.config("spark.executor.memory", "10G")\
.config("spark.driver.memory", "10G")\
.config("spark.memory.fraction", ".8")\
.getOrCreate()

In [125]:
defun = spark.read.csv('/home/jovyan/work/datos/defun_2016.csv', header =True) 
causas = spark.read.csv('/home/jovyan/work/datos/decatcausa.csv', header =True) 
estados = spark.read.csv('/home/jovyan/work/datos/decateml.CSV', header =True) 
genero = spark.read.csv('/home/jovyan/work/datos/desexo.csv', header =True) 
edad = spark.read.csv('/home/jovyan/work/datos/deedad.csv', header =True) 

In [88]:
for colum in defun.columns:
    defun = defun.withColumn(colum, regexp_replace(colum, '\t', '').cast("integer"))

In [134]:
edos_col = estados.columns
edos_col.remove('nom_loc')
estados1 = estados
for colum in edos_col:
    estados1 = estados1.withColumn(colum, regexp_replace(colum, '\t', '').cast("integer"))

In [89]:
num_rows = defun.count()

defun.select([(count(when(isnan(c) | col(c).isNull(), c))/num_rows).alias(c) for c in defun.columns]).toPandas()

,ent_regis,mun_regis,ent_resid,mun_resid,tloc_resid,loc_resid,ent_ocurr,mun_ocurr,tloc_ocurr,loc_ocurr,causa_def,lista_mex,sexo,edad,dia_ocurr,mes_ocurr,anio_ocur,dia_regis,mes_regis,anio_regis,dia_nacim,mes_nacim,anio_nacim,ocupacion,escolarida,edo_civil,presunto,ocurr_trab,lugar_ocur,necropsia,asist_medi,sitio_ocur,cond_cert,nacionalid,derechohab,embarazo,rel_emba,horas,minutos,capitulo,grupo,lista1,gr_lismex,vio_fami,area_ur,edad_agru,complicaro,dia_cert,mes_cert,anio_cert,maternas,lengua,cond_act,par_agre,ent_ocules,mun_ocules,loc_ocules,razon_m,dis_re_oax
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.939743,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [96]:
drop_list = ['maternas','gr_lismex','causa_def', 'lista_mex']
defun_c = defun.select([column for column in defun_c.columns if column not in drop_list])

In [98]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = defun_c.columns, outputCol = 'features')
output = assembler.transform(defun_c)

In [151]:
output.limit(10).toPandas()

,ent_regis,mun_regis,ent_resid,mun_resid,tloc_resid,loc_resid,ent_ocurr,mun_ocurr,tloc_ocurr,loc_ocurr,sexo,edad,dia_ocurr,mes_ocurr,anio_ocur,dia_regis,mes_regis,anio_regis,dia_nacim,mes_nacim,anio_nacim,ocupacion,escolarida,edo_civil,presunto,ocurr_trab,lugar_ocur,necropsia,asist_medi,sitio_ocur,cond_cert,nacionalid,derechohab,embarazo,rel_emba,horas,minutos,capitulo,grupo,lista1,vio_fami,area_ur,edad_agru,complicaro,dia_cert,mes_cert,anio_cert,lengua,cond_act,par_agre,ent_ocules,mun_ocules,loc_ocules,razon_m,dis_re_oax,features
0,1,3,1,1,15,1,1,1,15,1,2,4055,31,12,1993,26,10,2016,3,5,1938,11,1,3,8,8,88,9,1,1,2,1,99,8,8,20,30,2,8,37,8,1,16,8,99,10,2016,9,2,88,88,888,8888,0,999,"[1.0, 3.0, 1.0, 1.0, 15.0, 1.0, 1.0, 1.0, 15.0..."
1,1,1,1,6,8,1,1,1,15,1,2,4075,7,5,2010,12,2,2016,7,4,1935,11,3,5,8,8,88,2,1,4,1,1,3,8,8,15,19,9,4,67,8,1,20,8,99,2,2016,9,2,88,88,888,8888,0,999,"[1.0, 1.0, 1.0, 6.0, 8.0, 1.0, 1.0, 1.0, 15.0,..."
2,1,1,1,10,1,7777,1,10,1,7777,2,4070,3,11,2010,30,8,2016,16,6,1940,11,3,5,8,8,88,9,1,11,1,1,1,8,8,9,0,11,8,80,8,2,19,8,99,8,2016,9,2,88,88,888,8888,0,999,"[1.0, 1.0, 1.0, 10.0, 1.0, 7777.0, 1.0, 10.0, ..."
3,1,1,1,1,1,444,1,1,1,7777,2,4054,17,11,2007,9,9,2016,2,8,1953,11,1,5,8,8,88,2,1,11,3,1,1,9,9,5,0,4,2,52,8,2,15,9,99,9,2016,9,2,88,88,888,8888,0,999,"[1.0, 1.0, 1.0, 1.0, 1.0, 444.0, 1.0, 1.0, 1.0..."
4,1,6,1,6,8,1,1,6,8,1,2,4081,30,12,2015,8,1,2016,23,11,1934,11,9,5,1,2,9,9,1,11,3,1,2,8,8,17,30,20,25,103,8,1,21,8,30,12,2015,2,2,88,88,888,8888,0,999,"[1.0, 6.0, 1.0, 6.0, 8.0, 1.0, 1.0, 6.0, 8.0, ..."
5,1,1,1,1,15,1,1,1,15,1,1,4057,3,9,2015,12,1,2016,20,10,1957,2,5,5,8,8,88,2,1,11,3,1,1,8,8,17,20,4,2,52,8,1,16,8,3,9,2015,2,1,88,88,888,8888,0,999,"[1.0, 1.0, 1.0, 1.0, 15.0, 1.0, 1.0, 1.0, 15.0..."
6,1,1,1,1,15,1,1,1,15,1,2,4037,24,12,2015,6,1,2016,14,5,1978,11,4,4,8,8,88,9,1,11,3,1,2,9,9,6,30,4,2,52,8,1,12,9,24,12,2015,2,2,88,88,888,8888,0,999,"[1.0, 1.0, 1.0, 1.0, 15.0, 1.0, 1.0, 1.0, 15.0..."
7,1,1,1,1,15,1,1,1,15,1,2,4067,11,12,2015,12,1,2016,19,12,1947,11,4,3,8,8,88,9,1,1,3,1,7,8,8,23,2,9,3,66,8,1,18,8,12,12,2015,2,2,88,88,888,8888,0,999,"[1.0, 1.0, 1.0, 1.0, 15.0, 1.0, 1.0, 1.0, 15.0..."
8,1,1,1,1,15,1,1,1,15,1,1,4012,18,12,2015,4,1,2016,24,2,2003,11,4,1,8,8,88,2,1,1,1,1,7,8,8,13,35,6,9,61,8,1,7,8,18,12,2015,2,2,88,88,888,8888,0,999,"[1.0, 1.0, 1.0, 1.0, 15.0, 1.0, 1.0, 1.0, 15.0..."
9,1,1,1,1,15,1,1,1,15,1,1,4055,27,12,2015,5,1,2016,14,1,1960,7,6,5,8,8,88,2,1,4,3,1,3,8,8,20,0,2,11,42,8,1,16,8,27,12,2015,2,1,88,88,888,8888,0,999,"[1.0, 1.0, 1.0, 1.0, 15.0, 1.0, 1.0, 1.0, 15.0..."


In [117]:
kmeans = KMeans(k=32, seed=1)
model = kmeans.fit(output)

In [119]:
centers = model.clusterCenters()

In [162]:
centers[18]

array([  1.54873581e+01,   2.54161486e+01,   1.56613838e+01,
         2.78193637e+01,   1.09085081e+01,   9.28610369e+00,
         1.54826259e+01,   2.48235656e+01,   1.19117428e+01,
         5.92045585e+00,   1.48766723e+00,   4.06997830e+03,
         1.57261132e+01,   6.34200282e+00,   2.01596782e+03,
         1.65600411e+01,   6.29996723e+00,   2.01600000e+03,
         1.55825271e+01,   6.51770978e+00,   1.94550105e+03,
         9.26357510e+00,   6.43776329e+00,   3.80769569e+00,
         7.99734879e+00,   7.99891021e+00,   8.79694006e+01,
         2.93997351e+00,   1.46175736e+00,   7.87793265e+00,
         2.65085358e+00,   1.00524646e+00,   9.90748226e+00,
         7.98529447e+00,   7.92458853e+00,   1.20013456e+01,
         2.24890581e+01,   7.23775849e+00,   4.96442589e+00,
         6.15577683e+01,   8.00000533e+00,   1.13813979e+00,
         1.85987301e+01,   7.92499887e+00,   1.58946872e+01,
         6.33388400e+00,   2.01598583e+03,   2.71239085e+00,
         1.66271073e+00,

In [152]:
for ar in centers:
    print(ar[15].round())

18.0
21.0
18.0
26.0
42.0
25.0
17.0
19.0
33.0
18.0
19.0
35.0
38.0
17.0
31.0
18.0
24.0
45.0
17.0
40.0
37.0
27.0
19.0
18.0
37.0
16.0
19.0
46.0
23.0
38.0
25.0
17.0


In [138]:
estados1.filter('cve_mun == 0').toPandas()

,cve_ent,cve_mun,cve_loc,nom_loc
0,1,0,0,Aguascalientes
1,2,0,0,Baja California
2,3,0,0,Baja California Sur
3,4,0,0,Campeche
4,5,0,0,Coahuila de Zaragoza
5,6,0,0,Colima
6,7,0,0,Chiapas
7,8,0,0,Chihuahua
8,9,0,0,Ciudad de M�xico
9,10,0,0,Durango


In [183]:
transformed = model.transform(output)
#rows = transformed.collect()

In [179]:
summary = model.summary
groups = summary.cluster.withColumn('id', monotonically_increasing_id())

In [161]:
summary.clusterSizes[18]

375650

In [189]:
transformed.filter('prediction == 18').limit(10).toPandas()

,ent_regis,mun_regis,ent_resid,mun_resid,tloc_resid,loc_resid,ent_ocurr,mun_ocurr,tloc_ocurr,loc_ocurr,sexo,edad,dia_ocurr,mes_ocurr,anio_ocur,dia_regis,mes_regis,anio_regis,dia_nacim,mes_nacim,anio_nacim,ocupacion,escolarida,edo_civil,presunto,ocurr_trab,lugar_ocur,necropsia,asist_medi,sitio_ocur,cond_cert,nacionalid,derechohab,embarazo,rel_emba,horas,minutos,capitulo,grupo,lista1,vio_fami,area_ur,edad_agru,complicaro,dia_cert,mes_cert,anio_cert,lengua,cond_act,par_agre,ent_ocules,mun_ocules,loc_ocules,razon_m,dis_re_oax,features,prediction
0,1,3,1,1,15,1,1,1,15,1,2,4055,31,12,1993,26,10,2016,3,5,1938,11,1,3,8,8,88,9,1,1,2,1,99,8,8,20,30,2,8,37,8,1,16,8,99,10,2016,9,2,88,88,888,8888,0,999,"[1.0, 3.0, 1.0, 1.0, 15.0, 1.0, 1.0, 1.0, 15.0...",18
1,1,1,1,6,8,1,1,1,15,1,2,4075,7,5,2010,12,2,2016,7,4,1935,11,3,5,8,8,88,2,1,4,1,1,3,8,8,15,19,9,4,67,8,1,20,8,99,2,2016,9,2,88,88,888,8888,0,999,"[1.0, 1.0, 1.0, 6.0, 8.0, 1.0, 1.0, 1.0, 15.0,...",18
2,1,1,1,1,15,1,1,1,15,1,1,4057,3,9,2015,12,1,2016,20,10,1957,2,5,5,8,8,88,2,1,11,3,1,1,8,8,17,20,4,2,52,8,1,16,8,3,9,2015,2,1,88,88,888,8888,0,999,"[1.0, 1.0, 1.0, 1.0, 15.0, 1.0, 1.0, 1.0, 15.0...",18
3,1,1,1,1,15,1,1,1,15,1,2,4037,24,12,2015,6,1,2016,14,5,1978,11,4,4,8,8,88,9,1,11,3,1,2,9,9,6,30,4,2,52,8,1,12,9,24,12,2015,2,2,88,88,888,8888,0,999,"[1.0, 1.0, 1.0, 1.0, 15.0, 1.0, 1.0, 1.0, 15.0...",18
4,1,1,1,1,15,1,1,1,15,1,2,4067,11,12,2015,12,1,2016,19,12,1947,11,4,3,8,8,88,9,1,1,3,1,7,8,8,23,2,9,3,66,8,1,18,8,12,12,2015,2,2,88,88,888,8888,0,999,"[1.0, 1.0, 1.0, 1.0, 15.0, 1.0, 1.0, 1.0, 15.0...",18
5,1,1,1,1,15,1,1,1,15,1,1,4012,18,12,2015,4,1,2016,24,2,2003,11,4,1,8,8,88,2,1,1,1,1,7,8,8,13,35,6,9,61,8,1,7,8,18,12,2015,2,2,88,88,888,8888,0,999,"[1.0, 1.0, 1.0, 1.0, 15.0, 1.0, 1.0, 1.0, 15.0...",18
6,1,1,1,1,15,1,1,1,15,1,1,4055,27,12,2015,5,1,2016,14,1,1960,7,6,5,8,8,88,2,1,4,3,1,3,8,8,20,0,2,11,42,8,1,16,8,27,12,2015,2,1,88,88,888,8888,0,999,"[1.0, 1.0, 1.0, 1.0, 15.0, 1.0, 1.0, 1.0, 15.0...",18
7,1,1,1,1,15,1,1,1,15,1,1,4050,25,12,2015,5,1,2016,2,1,1965,4,3,5,8,8,88,2,1,11,3,1,7,8,8,5,10,11,8,80,8,1,15,8,25,12,2015,2,1,88,88,888,8888,0,999,"[1.0, 1.0, 1.0, 1.0, 15.0, 1.0, 1.0, 1.0, 15.0...",18
8,1,1,1,1,15,1,1,1,15,1,1,4063,28,12,2015,11,1,2016,2,6,1952,11,4,2,8,8,88,2,1,11,3,1,2,8,8,17,30,4,2,52,8,1,17,8,28,12,2015,2,2,88,88,888,8888,0,999,"[1.0, 1.0, 1.0, 1.0, 15.0, 1.0, 1.0, 1.0, 15.0...",18
9,1,1,1,1,15,1,1,1,15,1,2,4085,30,12,2015,5,1,2016,28,1,1930,11,1,1,8,8,88,2,1,11,3,1,2,8,8,6,45,9,7,69,8,1,22,8,30,12,2015,2,2,88,88,888,8888,0,999,"[1.0, 1.0, 1.0, 1.0, 15.0, 1.0, 1.0, 1.0, 15.0...",18
